In [35]:
from models.processing import filter_dataset, load_dataset
from dataclasses import dataclass
from typing import List
import pandas as pd
from gurobipy import *

In [14]:
def load_dataset(data_input):
    """
    Load dataset from the preprocessed CSV
    :return:
    """
    df = pd.read_csv(data_input)
    return df

In [40]:
def generate_distance_matrix(locations):
    """
    Given a list of n tuples representing location coordinates, return an nxn matrix containing all of the distances
    :param locations: list of n tuples representing location coordinates
    :return: nxn matrix containing all of the distances
    """
    return [[5 for i in locations] for j in locations]

In [2]:
@dataclass
class Bar:
    name: str
    latitude: str
    longitude: str
    rating: float

In [3]:
@dataclass
class Solution:
    bars = List[Bar]
    total_max_walking_time = float

In [50]:
def get_optimal_route(df, start_time, end_time, bar_num, total_max_walking_time, max_walking_each):
    """

    :param df:
    :param start_time:
    :param end_time:
    :param bar_num:
    :param total_max_walking_time:
    :param max_walking_each:
    :return: A Solution class object representing the optimal solution
    """
    
    #parameters
    m=Model("opt_route")
    locations=df['name']
    ratings=df['stars']
    y=[]
    x=[[0 for j in range(len(locations))] for i in range(len(locations))]
    z=[[0 for k in range(bar_num)] for i in range (len(locations))]
    dima=generate_distance_matrix(locations)
    waittimes=wait_time(df)
    
    #create decision variables
    for loc in locations:
        y.append(m.addVar(vtype=GRB.BINARY, name="y_{}".format(loc)))
    
    for i in range(len(locations)):
        for j in range(len(locations)):
            x[i][j]=m.addVar(vtype=GRB.BINARY,name="x_{},{}".format(locations[i],locations[j]))
            
    for i in range(len(locations)):
        for k in range(bar_num):
            z[i][k]=m.addVar(vtype=GRB.BINARY,name="x_{},{}".format(locations[i],k))
     
    #objective function
    m.setObjective(quicksum([y[i]*ratings[i] for i in range(len(locations))]),GRB.MAXIMIZE)
    
    #constraints
        #inbound arcs
    for j in range(len(locations)):
        m.addConstr(quicksum([x[i][j] for i in range(len(locations))])==y[j])
        
        #outbound arcs
    for i in range(len(locations)):
        m.addConstr(quicksum([x[i][j] for j in range(len(locations))])==y[i])
    
        #subtours
    
        #Locations visited
    m.addConstr(quicksum(y)==bar_num)
    
        #max total walk time
    m.addConstr(quicksum([x[i][j]*dima[i][j] for i in range(len(locations)) for j in range (len(locations))])<=total_max_walking_time)  
    
        #max walk time between locations
    for i in range(len(locations)):
        for j in range(len(locations)):
            m.addConstr(x[i][j]*dima[i][j]<=max_walking_each)
    
        #rules about z
    for k in range(bar_num):
        m.addConstr(quicksum([z[i][k] for i in range(len(locations))])==1)
                    
    for i in range(len(locations)):
        m.addConstr(quicksum([z[i][k] for k in range(bar_num)])<=1)
    
        #open  time
    for zed in range(bar_num):
        m.addConstr(start_time
                    +quicksum([z[i][k]*dima[i][j] for i in range(bar_num) for j in range(bar_num) for k in range(zed+1)])
                    +quicksum()
     
    m.setParam('TimeLimit', 30)
    m.setParam('MIPFocus',1)
    m.optimize()
    return m,z,y,x

In [5]:
def get_pareto_routes(df, start_time, end_time, bar_num, total_max_walking_time, max_walking_each):
    """
    Returns total_max_walking_time / 5 suggested routes (e.g. one for 0-5 mins walk, one for 5-10 mins walk, etc.).
    Pareto routes contain
    :param df:
    :param start_time:
    :param end_time:
    :param bar_num:
    :param total_max_walking_time:
    :param max_walking_each:
    :return: A list of Solutions
    """
    solutions = []
    for max_time in range(0, int(total_max_walking_time), 5):
        solutions.append(get_optimal_route(df, start_time, end_time, bar_num, max_time, max_walking_each))
    pass

In [6]:
def crawl_model(date, start_time, end_time, budget, bar_num,  total_max_walking_time,  max_walking_each, min_review_ct,
                min_review, city):
    """
    :param date:
    :param start_time:
    :param end_time:
    :param budget:
    :param bar_num:
    :param total_max_walking_time:
    :param max_walking_each:
    :param min_review_ct:
    :param min_review:
    :param city:
    :return:
    """

    df = load_dataset()
    df = filter_dataset(df, min_review_ct, min_review, date, budget, city)
    pareto_df = get_pareto_routes(df, start_time, end_time, bar_num, total_max_walking_time, max_walking_each)

    return pareto_df

In [16]:
toronto_data=load_dataset('data\\business_example.csv')

In [24]:
for loc in toronto_data["name"][:10]:
    print(loc)

Maman
The Shameful Tiki Room
Brass Taps Pizza Pub
Shallow Groove Lounge
Hogtown Pub & Oysters
Stirling Room
Panafest
Bathurst Local
Fiction Nightclub
The Pickle Barrel


In [28]:
strs=toronto_data['stars']
strs[187]

3.0

In [51]:
wait_time=generate_distance_matrix
optrout,z,y,x=get_optimal_route(toronto_data[:50], 12, 17, 6, 60, 10,45)

Changed value of parameter TimeLimit to 30.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter MIPFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Optimize a model with 2658 rows, 2850 columns and 10750 nonzeros
Variable types: 0 continuous, 2850 integer (2850 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [2e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective 19.0000000
Presolve removed 2557 rows and 300 columns
Presolve time: 0.02s
Presolved: 101 rows, 2550 columns, 5150 nonzeros
Variable types: 0 continuous, 2550 integer (2550 binary)

Root relaxation: objective 2.850000e+01, 106 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      28.5000000   28.50000  0.00%     -    0s

Explored 0 nodes (106

In [55]:
print([i for i in y if i.x==1])

[<gurobi.Var y_Another Bar (value 1.0)>, <gurobi.Var y_I'll Be Seeing You (value 1.0)>, <gurobi.Var y_Koerner Hall (value 1.0)>, <gurobi.Var y_King's Belly English Pub (value 1.0)>, <gurobi.Var y_Locus144 (value 1.0)>, <gurobi.Var y_Koricancha Restaurant (value 1.0)>]


In [1]:
list(range(0))

[]

In [3]:
mat=[[1,2,3],
    [4,5,6],
    [7,8,9]]
biny=[0,1,0]

In [7]:
[biny[j]*row[j] for row in [biny[i]*mat[i] for i in range(len(biny))]]

NameError: name 'j' is not defined

In [5]:
0*[1,2,3]

[]